In [20]:
import gpxpy
import matplotlib.pyplot as plt
import datetime
from geopy import distance
from math import sqrt, floor
import numpy as np
import pandas as pd
import haversine

# .gpx 파일 호출 후 파싱
gpx_file = open('practice.gpx', 'r')
gpx = gpxpy.parse(gpx_file)
len(gpx.tracks)
len(gpx.tracks[0].segments)
len(gpx.tracks[0].segments[0].points)
data = gpx.tracks[0].segments[0].points

start = data[0]
finish = data[-1]

# 경도, 위도, 높이, 시간
df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
for point in data:
    df = df.append({'lon': point.longitude,
                    'lat' : point.latitude,
                    'alt' : point.elevation,
                    'time' : point.time}, ignore_index=True)
df

C:\Users\USER\AppData\Local\Temp\ipykernel_2876\542953579.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'lon': point.longitude,
C:\Users\USER\AppData\Local\Temp\ipykernel_2876\542953579.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'lon': point.longitude,
C:\Users\USER\AppData\Local\Temp\ipykernel_2876\542953579.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'lon': point.longitude,
C:\Users\USER\AppData\Local\Temp\ipykernel_2876\542953579.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'lon': point.longitude,
C:\Users\USER\AppData\Local\Temp\ipykern

,lon,lat,alt,time
0,100.491491,13.725929,9.8,2018-07-24 07:32:31+00:00
1,100.491541,13.725939,10.5,2018-07-24 07:32:33+00:00
2,100.491575,13.725943,10.7,2018-07-24 07:32:35+00:00
3,100.491629,13.725970,10.8,2018-07-24 07:32:37+00:00
4,100.491667,13.725987,11.1,2018-07-24 07:32:38+00:00
...,...,...,...,...
2421,100.491484,13.726074,9.4,2018-07-24 08:29:15+00:00
2422,100.491439,13.726048,9.0,2018-07-24 08:29:17+00:00
2423,100.491406,13.726039,8.4,2018-07-24 08:29:18+00:00
2424,100.491352,13.726029,7.9,2018-07-24 08:29:20+00:00


In [22]:
# 사용할 변수 초기화
alt_dif = [0]
time_dif = [0]
dist_vin = [0]
dist_hav = [0]
dist_vin_no_alt = [0]
dist_hav_no_alt = [0]
dist_dif_hav_2d = [0]
dist_dif_vin_2d = [0]

# 사용할 변수의 데이터 제작
for index in range(len(data)): # 2426
    if index == 0:
        pass
    else:
        start = data[index-1] # 첫째행
        stop = data[index] # 둘째행
        
        # 1. 2차원 거리(경위도 공식(Vincenty formula))
        distance_vin_2d = distance.geodesic((start.latitude, start.longitude), (stop.latitude, stop.longitude)).m
        dist_dif_vin_2d.append(distance_vin_2d)
        # 2. 2차원 거리(하버사인 공식)
        distance_hav_2d = haversine.haversine((start.latitude, start.longitude), (stop.latitude, stop.longitude))*1000
        dist_dif_hav_2d.append(distance_hav_2d)
        
        # 3. 구간별 거리의 합
        dist_vin_no_alt.append(dist_vin_no_alt[-1] + distance_vin_2d)
        dist_hav_no_alt.append(dist_hav_no_alt[-1] + distance_hav_2d)
        
        # 4. 높이 차이
        alt_d = start.elevation - stop.elevation
        alt_dif.append(alt_d)
        
        # 5. 3차원 거리(경위도 공식(Vincenty formula))
        distance_vin_3d = sqrt(distance_vin_2d**2 + (alt_d)**2)
        # 6. 3차원 거리(하버사인 공식)
        distance_hav_3d = sqrt(distance_hav_2d**2 + (alt_d)**2)
        
        # 7. 시간 차이
        time_delta = (stop.time - start.time).total_seconds()
        time_dif.append(time_delta)
        
        # 8. 구간별 거리의 합
        dist_vin.append(dist_vin[-1] + distance_vin_3d)
        dist_hav.append(dist_hav[-1] + distance_hav_3d)

df['dis_vin_2d'] = dist_vin_no_alt # 2차원 vin 구간별 거리의 합 
df['dist_hav_2d'] = dist_hav_no_alt # 2차원 하버사인 구간별 거리의 합
df['dis_vin_3d'] = dist_vin # 3차원 vin 구간별 거리의 합 
df['dis_hav_3d'] = dist_hav # 3차원 하버사인 구간별 거리의 합
df['alt_dif'] = alt_dif # 높이차이
df['time_dif'] = time_dif # 시간 차이
df['dis_dif_hav_2d'] = dist_dif_hav_2d # 2차원 하버사인 구간별 거리
df['dis_dif_vin_2d'] = dist_dif_vin_2d # 2차원 vin 구간별 거리
df['dist_dif_per_sec'] = df['dis_dif_hav_2d'] / df['time_dif'] # 2차원 하버사인 구간별 속도(2차원 하버사인 구간별 거리 / 시간 차이)
df['spd'] = (df['dis_dif_hav_2d'] / df['time_dif']) * 3.6 # m/s -> km/h
df

,lon,lat,alt,time,dis_vin_2d,dist_hav_2d,dis_vin_3d,dis_hav_3d,alt_dif,time_dif,dis_dif_hav_2d,dis_dif_vin_2d,dist_dif_per_sec,spd
0,100.491491,13.725929,9.8,2018-07-24 07:32:31+00:00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN
1,100.491541,13.725939,10.5,2018-07-24 07:32:33+00:00,5.520049,5.514254,5.564256,5.558506,-0.7,2.0,5.514254,5.520049,2.757127,9.925657
2,100.491575,13.725943,10.7,2018-07-24 07:32:35+00:00,9.224049,9.213753,9.273651,9.263408,-0.2,2.0,3.699499,3.703999,1.849750,6.659099
3,100.491629,13.725970,10.8,2018-07-24 07:32:37+00:00,15.784300,15.774099,15.834664,15.824516,-0.1,2.0,6.560346,6.560251,3.280173,11.808623
4,100.491667,13.725987,11.1,2018-07-24 07:32:38+00:00,20.304312,20.293192,20.364621,20.353556,-0.3,1.0,4.519093,4.520012,4.519093,16.268736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421,100.491484,13.726074,9.4,2018-07-24 08:29:15+00:00,11055.923137,11068.928851,11125.593101,11138.569004,-0.1,2.0,3.883193,3.874669,1.941596,6.989747
2422,100.491439,13.726048,9.0,2018-07-24 08:29:17+00:00,11061.576856,11074.584505,11131.260953,11144.238786,0.4,2.0,5.655654,5.653719,2.827827,10.180178
2423,100.491406,13.726039,8.4,2018-07-24 08:29:18+00:00,11065.282448,11078.286963,11135.014806,11147.989545,0.6,1.0,3.702458,3.705593,3.702458,13.328849
2424,100.491352,13.726029,7.9,2018-07-24 08:29:20+00:00,11071.226992,11084.225056,11140.980340,11153.948651,0.5,2.0,5.938093,5.944544,2.969046,10.688567


In [23]:
# 이상치 제거
df_with_timeout = df[df['dist_dif_per_sec'] > 0.9] # 초당 발걸음이 0.9m 이내가 정상, 나머지 삭제

# 평균속도 = (속도*시간)의 총합/시간의 총합
avg_km_h = (sum((df_with_timeout['spd'] * df_with_timeout['time_dif'])) / sum(df_with_timeout['time_dif'])) # 11.865687875522866

# 1km를 평균속도로 이동시 걸린 시간
print(floor(60 / avg_km_h), 'minutes',round(((60 / avg_km_h - floor(60 / avg_km_h))*60), 0),' seconds')

5 minutes 3.0  seconds
